# Mini RNN  adapt for FPGA for very high frequency physics

## Introduction

### LHC, ATLAS AND Liquid Argon Detector

#### LHC


The CERN is operating the LHC (Large Hadron Collider). The LHC is the end of several particle accelerators and can accelerate particles to 7-8 GeV. There are four collision points on the LHC, ATLAS, CMS, LHCb and ALICE. The objective is to collide bench of particles into each other and create for a really short instant a really heavy particle (as E=mc²). Those heavy particles are so instable that it is not possible for us to detect them, however they disintegrate into smaller particles from which we can establish equation and deduct what was the big particle. 

The LHC is in constant improvement. In June, Run 3 of the LHC will occur. For this run, LHC is still on his first phase of development. 
But in the beginning in 2027, the HL-LHC will be hold. In this phase, LHC will become HL-LHC for High Luminosity. In fact, global improvement on the accelerator will give the possibility to greatly increase the number of particle crossing each other at each bench, this way, increasing the number of collision and data. 

The increased of the data amount has to be taken in account by the detector, this is why each of ATLAS detector acquisition system have to be improved. 


#### ATLAS

The CPPM is associated with ATLAS detector which is a photon-photon detector.  Atlas is built like a Russian-doll, with detector surrounding each other.

Beams of particles from the LHC collide at the centre of the ATLAS detector making collision debris in the form of new particles, which fly out from the collision point in all directions. Six different detecting subsystems arranged in layers around the collision point record the paths, momentum, and energy of the particles, allowing them to be individually identified. A huge magnet system bends the paths of charged particles so that their momenta can be measured.

The interactions in the ATLAS detectors create an enormous flow of data. To digest the data, ATLAS uses an advanced “trigger” system to tell the detector which events to record and which to ignore. Complex data-acquisition and computing systems are then used to analyse the collision events recorded. At 46 m long, 25 m high and 25 m wide, the 7000-tonne ATLAS detector is the largest volume particle detector ever constructed. It sits in a cavern 100 m below ground near the main CERN site, close to the village of Meyrin in Switzerland.

The project I will work on is dedicated for the Liquid Argon calorimeter. 

#### Liquid Argon Detector

The LAr is a calorimeter, it measures the energy. 

So how does the LAr acquisition system works ?  First, it is in constant improvement, and to be sure that the detector is always working, ATLAS team are always keeping the oldest working version on while testing a new version.

So basically the detector is composed of two plates with high voltage set inside a bath of Argon. The system is simple, every a particle goes across the bath, they transform Argon into ions, which create current in the system that can be measured. The more energy the particle has, the more the intensity is high. From the highest energy pick detected during the passage of the particle, we can deduce which particle it is. 

Now what is happening is that the acquisition system has to sample at 40MHz in order to follow the crazy rate of 1 collision every 25ns. This data can be stored for a short time on a buffer, but need to be sorted as soon as possible. To select the interesting part, a trigger is used. This means that we select only five inputs, that sums up the whole passage of the particle. Those five inputs are currently selected via an OF, this OF is encoded on FPGA. 

The current system is working well but struggles when two particles are crossing the detectors in the same time. 
If this is happening, a pile up phenomena occurs, and instead of having a clear pick of energy to detect, the two energies are mixed up by the detector and the trigger don’t know anymore how to work.

However, with the HL LHC, those events that were by the past rare, will become very probable. 

The main mission of the team is then to create a system that first can detect a pile up event, and then can evaluate the energy of each by separating the two events. 

To do so, RNN and CNN are being tested but those two have to fit on the FPGA montage that will be implemented at the end. 

The team at CPPM is focusing on the RNN work. They first came with an LSTM cell that repeat itself, this was unfortunately really efficient but is not possibly implantable on FPGA. They are now on vanilla cell but with a sliding windows. This solution seems to be more optimized but is still too big for FPGA. 

To fit the FPGA, what has already been done, is that they are using fixed quantized number instead of 32 bits long float number in the original code, but then efficiency is lost. 

What they want me to do is now, instead of taking a neural network trained and quantized it during the implementation of the FPGA, they want me to train a neural network already quantized. 


### Quantization

### Recurrent neural network

#### Neural Networks

A neural network (NN), is an interconnected group of natural or artificial neurons that uses a mathematical or computational model for information processing based on a connectionistic approach to computation. In most cases an NN is an adaptive system that changes its structure based on external or internal information that flows through the network.

##### Structure of the NN

The NN are composed of multiples layers. Each network is composed of one input layer, one output layer, and most of the time several 'hiddens' layers. 

These layers are composed of cells/units/nodes. Each of this nodes takes inputs and passes them into a function named the activation function. The activation function is the same for the whole layer, but in each unit, the input is implemented with differents weights. 

Those weights shape the input of the activation function and so the output as they help determine the importance of any given variable, with larger ones contributing more significantly to the output compared to other inputs.


The outputs of the node is then going to all the cells of the next layer and this is how the data go throught the network until it reaches the output layer.


#### The training

The outputs are shaped all along the network by the successives activations functions. Those functions are themselves altered by the weights. 
Now what is interesting is to see how those weights are chosen to get to the good outputs.
In fact those weigths are the final step of a long adjustement call the training.

During this step, the weights are optimized to maximize the neural network's accuracy.

Although, the weights are highly interdependent, indeed, the choice of one weight will have direct consequences on every nodes of the next layer and so on evry weights of n+2 layer. 
So we can not obtain the best weight by optimizing them one by one. Then we need to find the best combinaison of weight.

For simple problem we can use linear regression, where we try to create line that passes the closest to the most points : this is measured with the loss function that in this case calculate the mean distance between the line and the points. 
Then we can use gradient descent to find one point that minimize enought this loss function. 

Although, most NN problems don't have linear problem. This is why we are using nonlinear activation function. 
This nonlinearity means that the parameters do not act independantly of each other in influencing the shape of the loss function.

The loss will not have anymore a linear curve that allows us to find out the minimal just by running gradient descent. 


##### Gradient descent

Intuitively, the way gradient descent works is similar to the mountain climber analogy, were the climber in the dark check for every direction and choose the one that seems to go the most down. 
We first start with a random guess and then step by step we repeat the same process until we found the deepest point.
Then, what need to be decided is how large the step have to be. 

## Tests 

### Tests on RNN

On Keras model I want to determine the impact of the following features :

#### Patiences Early stopping and patiences for Reduce LR :


Increasing the patience will let more chances to the model to improve find a way out of a local minimal.
Reducing the patience reduces the training time.

From what I oserved it seems that once the system is stuck at 5 epochs without improving, it is never improving more.
The patiences for early stopping a have to be related to the patience of the Reduce LR. 
It has to be greater so that, if not reduce LR will never be used.

Indeed the patience for reduce lr is waiting for the model to  be stucked somewhere it can't improved to reduce the learning rate and check to improved it.

I am testing to set those to 8 and 5, this allows the model to be stuck during 8 epochs, at the fifth one, he is reducing the lr, if nothings happened after three more epochs, then we stop the learning.

Regarding the reference model, trained on a fixed 0.0001 lr, we cna say that 
After observations, it seems that the threshold to get to anew 

#### Batch size

#### Learning rate 

The learning rate here is not this important as he is evolving with the reduced lr callback.

### How to generate the ROOTs plots

Run this and generate a .ROOT file

In [ ]:
import importlib
import nnDumper_standalone
importlib.reload(nnDumper_standalone)

ds = DataShaper.from_h5("data/rdgap_mu140.h5")
x, x_val, x_test, y, y_val, y_test = ds()
m_t = 16
model = qkeras.utils.load_qmodel('path_to_model')
dump_preds = nnDumper_standalone.dumper(start = 999997, name = "test", bt_len=80)

dump_preds.set_data(true = ds.hit, data = ds.dig, sig = ds.sig, ofmax = ds.ofmax)

dump_preds.set_preds('type', m_t*model.predict(x_test).flatten())

dump_preds.runme()

In this file, resolution_drawer_with_gaps.C define the type of your model and set a label to it. 
You also need to file the path of the input (look for 'infiles'). 
It is also required to change the file to compare (just below the infile path)

Then setup the environement by running those :

export ATLAS_LOCAL_ROOT_BASE=/cvmfs/atlas.cern.ch/repo/ATLASLocalRootBase
alias setupATLAS='source ${ATLAS_LOCAL_ROOT_BASE}/user/atlasLocalSetup.sh'
export ALRB_rootVersion=6.14.04-x86_64-slc6-gcc62-opt

setupATLAS
lsetup root

Generate the plots by running these : 
`echo root-config --cflags --glibs` | sed s/" -std=c++11"//g
g++ resolution_drawer_with_gaps.C -o resdrawer -lm -g -Wall ADD_HERE_OUTPUT_OF_PREVIOUS_LINE
./resdrawer 2>&1 | tee log.resdrawer
